# Mosaic
Inputs:
- goal image
- folder of images
- number of photos you want in image

Output
- image

In [5]:
INPUT_IMAGE_PATH = "data/targetimages/boots.JPG"
OUTPUT_IMAGE_PATH = "data/outputimages/woop250photos-200pixels-60opacity.jpg"
ALBUM_FOLDER_PATH = "data/albums/boots/"
ALBUM_PHOTO_EXTENSION = "*.JPG"


# Step 1: Index Photos


In [6]:
from PIL import Image
from IPython.display import clear_output
import numpy as np

from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

import glob
import json

In [118]:
class DistanceHeuristics:
    def __init__(self, indexPath):
        with open(indexPath, "r") as f:
            data = f.read()
            self.photos = json.loads(data)
            print("Photos index loaded.")

        self.lookup = np.ndarray((256,256,256,1), dtype="int")
    
    def deltaCIE(self, inputColor):
        color2_rgb = sRGBColor(inputColor[0], inputColor[1], inputColor[2])
        # Convert from RGB to Lab Color Space
        color2_lab = convert_color(color2_rgb, LabColor)

        delta = {
            "path": "none",
            "amount": 10000000
        }

        for photo in self.photos:
            photoRGB = eval(photo['dominantColor'])
            color1_rgb = sRGBColor(photoRGB[0], photoRGB[1], photoRGB[2])

            # Convert from RGB to Lab Color Space   
            color1_lab = convert_color(color1_rgb, LabColor)

            # Find the color difference
            delta_e = delta_e_cie2000(color1_lab, color2_lab)

            if delta_e < delta['amount']:
                delta['amount'] = delta_e
                delta['path'] = photo['fileName']
        
        return delta['path']
    
    def deltaCIEOptimized(self, inputColor):
        
        # OPTIMIZATION: check to see if input color has already been searched for
        if self.lookup[inputColor[0]][inputColor[1]][inputColor[2]][0] != 0:
            # print("Input color exists!")
            # print(inputColor)
            # print(self.lookup[inputColor[0]][inputColor[1]][inputColor[2]])
            idx = self.lookup[inputColor[0]][inputColor[1]][inputColor[2]][0]
            return self.photos[idx]['fileName']

        color2_rgb = sRGBColor(inputColor[0], inputColor[1], inputColor[2])
        # Convert from RGB to Lab Color Space
        color2_lab = convert_color(color2_rgb, LabColor)

        delta = {
            "path": "none",
            "amount": 10000000
        }

        enumPhotos = enumerate(self.photos)

        for i, photo in enumPhotos:
            photoRGB = eval(photo['dominantColor'])
            color1_rgb = sRGBColor(photoRGB[0], photoRGB[1], photoRGB[2])

            # Convert from RGB to Lab Color Space   
            color1_lab = convert_color(color1_rgb, LabColor)

            # Find the color difference
            delta_e = delta_e_cie2000(color1_lab, color2_lab)

            if delta_e < delta['amount']:
                delta['amount'] = delta_e
                delta['path'] = photo['fileName']
                delta['index'] = i
        
        self.lookup[inputColor[0]][inputColor[1]][inputColor[2]][0] = delta['index']
        return delta['path']

def getDominantColorImage(img, pixelLength):
    img = img.resize((1,1), resample=0)
    img = img.resize((pixelLength, pixelLength))
    return img


def getDominantColorFromImage(img):
    img.resize((1,1), resample=0)
    return img.getpixel((0,0))


In [8]:

def getPhotoPaths(path):
    return glob.glob(path)

def indexPhotoFolder(path, indexFileName, lim=None):
    photoPaths = getPhotoPaths(path)

    dominant_indexed = []

    for i in range(len(photoPaths[0:lim])):
        photo = Image.open(photoPaths[i])
        dominantColor = str(getDominantColorFromImage(photo))
        dominant_indexed.append({"fileName":str(photoPaths[i]),"dominantColor":dominantColor})
        
        # print("Photos Indexed: {}/{}".format(i+1, len(photoPaths[0:lim])) )

    f = open(indexFileName, "w")
    f.write(json.dumps(dominant_indexed))
    f.close()
    print("Done indexing photo folder!")
    return 

indexPhotoFolder(ALBUM_FOLDER_PATH + ALBUM_PHOTO_EXTENSION, ALBUM_FOLDER_PATH + 'indexed.txt')

Done indexing photo folder!


____________________________________________________________

# Step 2: Compute Image

In [120]:
strategies = ["dominantFromTarget", "dominantFromAlbum", "dominantFromAlbumWithBlend", "dominantFromAlbumWithBlendAndOptimized"]
selectedStrategyIndex = 3
downSizeFactor = 5 # make photo 5x smaller
subPhotoPixelLength = 200
numberSubPhotosHorizontal = 10

# Downsize image to make processing a bit easier. 
selectedPhoto = Image.open(INPUT_IMAGE_PATH)
initialPhotoWidth, initialPhotoHeight = selectedPhoto.size
selectedPhoto = selectedPhoto.resize((int(initialPhotoWidth / downSizeFactor), int(initialPhotoHeight / downSizeFactor)))
initialPhotoWidth, initialPhotoHeight = selectedPhoto.size

# Determine scale of new image
scaleFactor = int(np.floor(initialPhotoWidth/numberSubPhotosHorizontal))
numberSubPhotosVertical = int(np.floor(initialPhotoHeight / scaleFactor))
finalPhotoWidth = int(subPhotoPixelLength * numberSubPhotosHorizontal)
finalPhotoHeight = int(subPhotoPixelLength  * numberSubPhotosVertical)

finalPhoto = Image.new("RGB", (finalPhotoWidth, finalPhotoHeight))


def formatAlbumPhoto(photoPath, pixelLength, blend=False, alpha=0.5, dominantPhoto=False):
    photo = Image.open(photoPath)
    photo = photo.resize((pixelLength, pixelLength))
    if blend:
        return Image.blend(photo, dominantPhoto, alpha)
    return photo


def findBestMatchPhoto(cropPhoto, pixelLength):
    # Strat 0: Just get dominant color from original target image
    if (strategies[selectedStrategyIndex] == 'dominantFromTarget'):
        dominant_color = getDominantColorFromImage(cropPhoto)
        return Image.new('RGB', (pixelLength, pixelLength), dominant_color)
    
    if (strategies[selectedStrategyIndex] == "dominantFromAlbum"):
        dominantColorFromTarget = getDominantColorFromImage(cropPhoto)
        photoPath = D.deltaCIE(dominantColorFromTarget)
        photo = formatAlbumPhoto(photoPath, pixelLength)
        return photo
    
    if (strategies[selectedStrategyIndex] == "dominantFromAlbumWithBlend"):
        dominantColorFromTarget = getDominantColorFromImage(cropPhoto)
        photoPath = D.deltaCIE(dominantColorFromTarget)
        photo = formatAlbumPhoto(photoPath, pixelLength, True, 0.6, getDominantColorImage(cropPhoto, pixelLength))
        return photo
    
    if (strategies[selectedStrategyIndex] == "dominantFromAlbumWithBlendAndOptimized"):
        dominantColorFromTarget = getDominantColorFromImage(cropPhoto)
        photoPath = D.deltaCIEOptimized(dominantColorFromTarget)
        photo = formatAlbumPhoto(photoPath, pixelLength, True, 0.6, getDominantColorImage(cropPhoto, pixelLength))
        return photo

D = DistanceHeuristics(ALBUM_FOLDER_PATH + 'indexed.txt')

for i in range(numberSubPhotosHorizontal):
    for j in range(numberSubPhotosVertical):
        selectedPhotoCrop = selectedPhoto.crop((i * scaleFactor, j * scaleFactor, i * scaleFactor + scaleFactor, j * scaleFactor + scaleFactor))
        bestMatchPhoto = findBestMatchPhoto(selectedPhotoCrop, subPhotoPixelLength)
        finalPhoto.paste(bestMatchPhoto, box=(i * subPhotoPixelLength, j * subPhotoPixelLength))

        print(f"Completed {i}/{numberSubPhotosHorizontal}, {j}/{numberSubPhotosVertical}", end='\r')

finalPhoto.save(OUTPUT_IMAGE_PATH)


Photos index loaded.


# Random Tests, clean later

In [132]:
# given an image
img = Image.open('boots.JPG')
img = img.resize((50,50))
img2 = Image.open('boots.JPG')
img2 = img2.resize((50,50))
img3 = Image.blend(img, img2, .1)
img3.show()

# tint it slightly red


In [ ]:
def generateCollage(inputPhotoPath, inputPhotosFolderPath, outputPhotoPath, desiredResolution, allowRepeat=True):
    # if (inputPhotosFolderPath is not fully indexed)
        # index it

    # collageDimension = [50, 40] 50 pixels/images wide, 40 pixels/images tall
    # collageDimension = getCollageDimension(inputPhotoPath, desiredResolution)

    # generate photo

# from PIL import Image
# from IPython.display import clear_output

# CREATE COLLAGE
# selectedPhotoName = "boots.jpg"
# folderPath = "boots/"

# SUBPHOTO_WIDTH = 50
# SUBPHOTO_HEIGHT = SUBPHOTO_WIDTH
# SUBPHOTO_DIMENSIONS = (SUBPHOTO_WIDTH,SUBPHOTO_HEIGHT)

# FINAL_WIDTH_NUMPHOTOS, FINAL_HEIGHT_NUMPHOTOS = 100, 100
# FINAL_WIDTH, FINAL_HEIGHT = FINAL_WIDTH_NUMPHOTOS*SUBPHOTO_WIDTH, FINAL_HEIGHT_NUMPHOTOS*SUBPHOTO_HEIGHT


# collagePixels = Image.open(selectedPhotoName, "r")
# collagePixels = collagePixels.resize((FINAL_WIDTH_NUMPHOTOS,FINAL_HEIGHT_NUMPHOTOS),Image.ANTIALIAS)

# collagePhoto = Image.new("RGB",(FINAL_WIDTH,FINAL_HEIGHT))

# for i in range(FINAL_WIDTH_NUMPHOTOS):
#     for j in range(FINAL_HEIGHT_NUMPHOTOS):
#         clear_output(wait=True)
        
#         collagePixel = collagePixels.getpixel((i,j))
#         matchedSubPhotoPath = C.matchPixelToSubPhoto(collagePixel, indexPath='boots/index.txt')
        
#         subPhoto = C.getResizedSubPhoto(matchedSubPhotoPath, SUBPHOTO_DIMENSIONS)
#         collagePhoto.paste(subPhoto, box=(i*SUBPHOTO_WIDTH,j*SUBPHOTO_HEIGHT))
        
#         print("Photos Added to Collage: [{}/{}],[{}/{}]".format(i+1,FINAL_WIDTH_NUMPHOTOS, j+1, FINAL_HEIGHT_NUMPHOTOS))

# collagePhoto.save('output_image3.JPG')

# print("Done!")

In [96]:
lookup = np.ndarray((1,1,1,1), dtype="int")
print(lookup)
lookup[0][0][0][0] = 5
print("LOOKUP BELOE")
print(lookup[0][0][0][0])
print("LOOKUP ABOVE")

[[[[5]]]]
LOOKUP BELOE
5
LOOKUP ABOVE
